In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [8]:
magic = pd.read_csv('magic.txt', header=None,sep="\ \ ",engine='python')
magic.head()

,0,1,2,3,4,5,6,7,8,9,10
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1.0
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1.0
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1.0
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1.0
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1.0


Preparing X and y

In [53]:

temp = np.transpose(magic.values)
y = temp[-1]
y = y-1
temp = np.delete(temp,10,0)
#normalizing
for i,row in enumerate(temp):
    mean = np.mean(row)
    maxim = np.max(row)
    minim = np.min(row)
    temp[i] = (row - mean)/(maxim - minim)
X = temp.transpose()



In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

### Supervised Score

In [59]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.77733964248159837

Now splitting data to 